In [2]:
import numpy as np
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize,RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from nltk.sentiment.util import mark_negation
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
import re

In [2]:

def load_pickle(filename):
    with open(filename,"rb") as f:
        dump=pickle.load(f)
    return dump

def dump_pickle(dump,filename):
    with open(filename,"wb") as f:
        pickle.dump(dump,f)


In [3]:
train_file='data/A1_Data/toy.json'
dev_file='data/A1_Data/toy.json'
tokenized_train="tokenized_train.pkl"
tokenized_dev="tokenized_dev.pkl"
log=10**5

In [4]:
train_file='data/A1_Data/train.json'
dev_file='data/A1_Data/dev.json'

In [5]:
def read_data(filename):
    x=[]
    y=[]
    
    file_reader=open(filename,"r")
    for line in file_reader:
        mapping=json.loads(line)
        x.append(mapping['review'])
        y.append(mapping['ratings'])
    
    return x,y

In [6]:
train_sent,train_y=read_data(train_file)
dev_sent,dev_y=read_data(dev_file)
print(len(train_sent),len(dev_sent))

1000000 200000


## Tokenizer

In [39]:
def tokenize_data(data):
    tokenized=[]
    for i in range(len(data)):
        if(i%100000==0):
            print(i)
        tokenized.append(word_tokenize(data[i]))
    return tokenized

In [15]:
# train_x=tokenize_data(train_sent)
# dev_x=tokenize_data(dev_sent)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
0
100000


In [8]:
# print(len(train_x),train_x[0],type(train_x),tokenized_train)

In [9]:
# dump_pickle(train_x,tokenized_train)
# dump_pickle(dev_x,tokenized_dev)

In [152]:
word_tokenize("I am ANkesh. I am 21 :)")

['I', 'am', 'ANkesh', '.', 'I', 'am', '21', ':', ')']

## Loader

In [8]:
train_x=load_pickle(tokenized_train)
dev_x=load_pickle(tokenized_dev)

In [18]:
print(len(train_x),len(dev_x))

1000000 200000


In [49]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 
#Consider Stemming

In [50]:
def rem_stopwords_lemmatize(data):
    new_data=[]
    count=0
    for line in data:
        count+=1
        if(count%log==0):
            print(count)
        line=mark_negation(line,shallow=True,double_neg_flip=True)
        new_line=[]
        for word in line:
            lemmatized_word=lemmatizer.lemmatize(word)
            if lemmatized_word not in stop_words:
                new_line.append(lemmatized_word)
        new_data.append(new_line)
    return new_data

In [51]:
# train_x=rem_stopwords_lemmatize(train_x)
# dev_x=rem_stopwords_lemmatize(dev_x)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
100000
200000


## Save Cleaned

In [52]:
# dump_pickle(train_x,'cleaned_train.pkl')
# dump_pickle(dev_x,'cleaned_dev.pkl')mean_squared_error(y_true, y_pred)

## Load Cleaned

In [7]:
train_x_comp=load_pickle('cleaned_train.pkl')
dev_x_comp=load_pickle('cleaned_dev.pkl')

In [8]:
# cut=50000
# train_x=train_x_comp[0:cut]
# train_y=train_y[0:cut]
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regressiondev_x=dev_x_comp[0:cut]
# dev_y=dev_y[0:cut]

In [9]:
print(len(train_x_comp),len(dev_x_comp))
print(isinstance(train_x_comp[0][0],str))
for i in train_x_comp:
    for j in i:
        if(isinstance(j,str)==False):
            print(j)

1000000 200000
True


In [10]:
# Helper:https://www.ritchieng.com/machine-learning-multinomial-naive-bayes-vectorization/

## Prepare Data

In [11]:
def temp(x):
    return x

vect = CountVectorizer(analyzer=temp,ngram_range=(1, 2),max_df=0.85)
print(vect)
train_x_features = vect.fit_transform(train_x_comp)
dev_x_features = vect.transform(dev_x_comp)
train_x_features,dev_x_features

CountVectorizer(analyzer=<function temp at 0x7ff0b06f7d90>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.85,
        max_features=None, min_df=1, ngram_range=(1, 2), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)


(<1000000x663425 sparse matrix of type '<class 'numpy.int64'>'
 	with 63467831 stored elements in Compressed Sparse Row format>,
 <200000x663425 sparse matrix of type '<class 'numpy.int64'>'
 	with 12618646 stored elements in Compressed Sparse Row format>)

### TF-IDF

In [12]:
def temp(x):
    return x

vectorizer = TfidfVectorizer(analyzer=temp,ngram_range=(1,2))
vectorizer.fit(train_x_comp)
train_x_tfidf=vectorizer.transform(train_x_comp)
dev_x_tfidf=vectorizer.transform(dev_x_comp)
train_x_tfidf,dev_x_tfidf

(<1000000x663426 sparse matrix of type '<class 'numpy.float64'>'
 	with 64420201 stored elements in Compressed Sparse Row format>,
 <200000x663426 sparse matrix of type '<class 'numpy.float64'>'
 	with 12809187 stored elements in Compressed Sparse Row format>)

In [13]:
def get_accuracy(gold,pred):
    length=len(gold)
    return np.sum(gold==pred)*1.0/length
def get_Fscore(gold,pred):
    return (f1_score(gold, pred, average='macro'),f1_score(gold, pred, average='micro'))      

## General Fitter

In [14]:
def GeneralFitter(X,Y,obj):
    %time obj.fit(X,Y)
    return obj

## Models

In [42]:
nb=MultinomialNB()
clf = LinearSVC(random_state=0, tol=1e-5,verbose = 5,max_iter=1000,class_weight='balanced')
log_reg = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=100)
reg = LinearRegression()

In [15]:
def get_stats(X,Y,dev_X,dev_Y,obj):
    pred_X=obj.predict(X)
    pred_dev=obj.predict(dev_X)
    print(get_accuracy(Y,pred_X))
    print(get_accuracy(dev_y,pred_dev))
    print(get_Fscore(Y,pred_X))
    print(get_Fscore(dev_y,pred_dev))
    print(mean_squared_error(dev_y,pred_dev))
    print(confusion_matrix(Y, pred_X))
    print(confusion_matrix(dev_y, pred_dev))

In [16]:
def get_reg_stats(X,Y,dev_X,dev_Y,obj):
    pred_X=obj.predict(X)
    pred_dev=obj.predict(dev_X)
    pred_X[pred_X[:]<1]=1
    pred_X[pred_X[:]>5]=5
    pred_dev[pred_dev[:]<1]=1
    pred_dev[pred_dev[:]>5]=5
    
    print(mean_squared_error(Y,pred_X))    
    print(mean_squared_error(dev_y,pred_dev))

## Feature filtering

In [17]:
selection_obj=SelectKBest(chi2, k=1000)
filtered_train = selection_obj.fit_transform(train_x_features, train_y)
filtered_dev=selection_obj.transform(dev_x_features)
filtered_train,filtered_dev

(<1000000x1000 sparse matrix of type '<class 'numpy.int64'>'
 	with 33637287 stored elements in Compressed Sparse Row format>,
 <200000x1000 sparse matrix of type '<class 'numpy.int64'>'
 	with 6732487 stored elements in Compressed Sparse Row format>)

In [25]:
selection_obj_tfidf=SelectKBest(chi2, k=20000)
filtered_train_tfidf = selection_obj_tfidf.fit_transform(train_x_tfidf, train_y)
filtered_dev_tfidf=selection_obj_tfidf.transform(dev_x_tfidf)
filtered_train_tfidf,filtered_dev_tfidf

(<1000000x20000 sparse matrix of type '<class 'numpy.float64'>'
 	with 59052593 stored elements in Compressed Sparse Row format>,
 <200000x20000 sparse matrix of type '<class 'numpy.float64'>'
 	with 11816937 stored elements in Compressed Sparse Row format>)

In [19]:
print(filtered_dev_tfidf.shape)

(200000, 1000)


## Naive Bayes

In [34]:
nb_cv=MultinomialNB()
GeneralFitter(train_x_features,train_y,nb_cv)
get_stats(train_x_features,train_y,dev_x_features,dev_y,nb_cv)

CPU times: user 2.16 s, sys: 196 ms, total: 2.36 s
Wall time: 1.26 s
0.651809
0.613925
(0.5790082449946594, 0.651809)
(0.5249208163419372, 0.613925)
1.0594
[[112653  22560   7604   3551   3792]
 [ 20777  29410  19249   8788   3046]
 [ 10846   8410  48371  34821   8133]
 [  8539   3915  15745 133360  58069]
 [ 19802   2389   7104  81051 328015]]
[[21894  4700  1567   794   834]
 [ 4413  4439  4715  1868   704]
 [ 2263  2147  7518  8432  1756]
 [ 1791   758  3929 24605 12923]
 [ 4242   457  1307 17615 64329]]


In [39]:
# nb_tf=MultinomialNB()
# GeneralFitter(train_x_tfidf,train_y,nb_tf)
# get_stats(train_x_tfidf,train_y,dev_x_tfidf,dev_y,nb_tf)

## Logistic

In [46]:
log_reg_cv = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=100)
GeneralFitter(train_x_features,train_y,log_reg_cv)
get_stats(train_x_features,train_y,dev_x_features,dev_y,log_reg_cv)n_jobs=-1

CPU times: user 9min 5s, sys: 13.1 s, total: 9min 18s
Wall time: 5min 59s


/home/ankesh/anaconda3/envs/microsoft_AI/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

0.685439
0.67845
(0.5883533698893385, 0.685439)
(0.5775157339302541, 0.67845)
0.72456
[[124726  11827   3640   1953   8014]
 [ 26740  26873  16017   5790   5850]
 [  8869  12462  42375  32380  14495]
 [  3217   2633  15853  99295  98630]
 [  4201   1000   2854  38136 392170]]
[[24476  2481   770   405  1657]
 [ 5473  5066  3284  1173  1143]
 [ 1813  2620  8201  6578  2904]
 [  634   536  3227 19531 20078]
 [  892   196   585  7861 78416]]


In [38]:
log_reg_tf=LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=100)
GeneralFitter(train_x_tfidf,train_y,log_reg_tf)
get_stats(train_x_tfidf,train_y,dev_x_tfidf,dev_y,log_reg_tf)

/home/ankesh/anaconda3/envs/microsoft_AI/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


CPU times: user 7min 36s, sys: 11 s, total: 7min 47s
Wall time: 3min 41s
0.695181
0.685575
(0.6042086654923555, 0.695181)
(0.5910936223992282, 0.685575)
0.652865
[[127398  11249   4353   2167   4993]
 [ 26789  26095  18444   5884   4058]
 [  8576  10855  47807  33007  10336]
 [  3393   2127  18064 112784  83260]
 [  4633    718   3612  48301 381097]]
[[25040  2367   911   458  1013]
 [ 5457  4935  3804  1166   777]
 [ 1767  2334  9182  6754  2079]
 [  682   439  3753 21963 17169]
 [  964   160   777 10054 75995]]


In [40]:
dump_pickle(log_reg_tf,"log_reg_tf.pkl")

## SVM

In [45]:
svm = LinearSVC(random_state=0, tol=1e-5,verbose = 5,max_iter=1000,class_weight='balanced')
GeneralFitter(filtered_train_tfidf,train_y,svm)
get_stats(filtered_train_tfidf,train_y,filtered_dev_tfidf,dev_y,svm)

[LibLinear]CPU times: user 2min 28s, sys: 107 ms, total: 2min 28s
Wall time: 2min 30s
0.661501
0.66069
(0.577770452154133, 0.661501)
(0.576785221322023, 0.66069)
0.79196
[[123976  17182   3720   1293   3989]
 [ 26136  32515  15562   3878   3179]
 [  9833  19372  46441  25159   9776]
 [  5648   7626  28289  97713  80352]
 [  9501   4160   8976  54868 360856]]
[[24531  3417   768   281   792]
 [ 5245  6465  3042   770   617]
 [ 1984  3917  9260  4994  1961]
 [ 1149  1502  5655 19629 16071]
 [ 1920   876  1837 11064 72253]]


In [48]:
svm_full = LinearSVC(random_state=0, tol=1e-5,verbose = 5,max_iter=1000,class_weight='balanced')
GeneralFitter(train_x_tfidf,train_y,svm_full)
get_stats(train_x_tfidf,train_y,dev_x_tfidf,dev_y,svm_full)

[LibLinear]CPU times: user 3min 37s, sys: 204 ms, total: 3min 37s
Wall time: 3min 45s
0.811779
0.66543
(0.7812145589227063, 0.811779)
(0.5818336961995876, 0.66543)
0.657005
[[136987   9501   2438    514    720]
 [ 12369  59743   7159   1330    669]
 [  4658   9498  80853  12052   3520]
 [  2051   4115  16965 147496  49001]
 [  2536   2090   6556  40479 386700]]
[[24145  3877  1058   275   434]
 [ 4946  6379  3612   816   386]
 [ 1681  4066  9603  5380  1386]
 [  694  1589  6203 20748 14772]
 [ 1041   758  1956 11984 72211]]


## Linear Reg

In [26]:
reg = LinearRegression(n_jobs=-1)
GeneralFitter(filtered_train_tfidf,train_y,reg)

CPU times: user 14min 45s, sys: 724 ms, total: 14min 46s
Wall time: 3min 42s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [27]:
get_reg_stats(filtered_train_tfidf,train_y,filtered_dev_tfidf,dev_y,reg)

0.5577533438926271
0.5874714013902615


In [29]:
ridge=linear_model.Ridge(alpha=4)
GeneralFitter(filtered_train_tfidf,train_y,ridge)
get_reg_stats(filtered_train_tfidf,train_y,filtered_dev_tfidf,dev_y,ridge)

CPU times: user 24.9 s, sys: 312 ms, total: 25.2 s
Wall time: 23.7 s
0.5737841644944517
0.5900407214194511


In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression

In [48]:
# regr = RandomForestRegressor(max_depth=5, random_state=0,n_estimators=10,n_jobs=-1)
# GeneralFitter(filtered_train_tfidf,train_y,regr)
# get_reg_stats(filtered_train_tfidf,train_y,filtered_dev_tfidf,dev_y,regr)

In [49]:
word_tokenize("i didn't like movie")

['i', 'did', "n't", 'like', 'movie']

In [51]:
mark_negation(word_tokenize("i didn't, like movie"))

['i', 'did', "n't", ',_NEG', 'like_NEG', 'movie_NEG']